In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import pymc3 as pm

#MCMC settings
chain_length = 1_000
burnin = 500

#Observed I and SigI
I = [-3., 0., 1., 3.]
SigI =  1.

#Scale factor
Sigma = 1.

#Gamma prior params
#Gamma(J|alpha,beta) ∝ J^(alpha-1) * exp(-beta * J)
alpha = 1.0
beta = 1. / Sigma

with pm.Model() as model:
    #Sample from the prior P(J)
    J = pm.distributions.Gamma('Wilson', alpha, beta, shape=len(I))
    #Notice this distribution has an `observed` keyword argument indicating this is where data enters the model
    likelihood = pm.distributions.Normal('Likelihood', mu=J, sigma=SigI, observed=I)
    #The trace object will store the posterior samples
    trace = pm.sample(draws=chain_length, tune=burnin)

#These are samples from the posterior P(J|I)
samples = trace.get_values('Wilson')
data = pd.DataFrame({
    "J" : samples.flatten(),
    "I" : (np.ones_like(samples)*I).flatten(),
})


sns.histplot(x='J', data=data, hue='I', palette='Set2', stat='density')
plt.title("Posterior P(J|I)")

#Compute the posterior mean and uncertainty from the samples
data.groupby('I').apply(lambda x: pd.Series({'$\langle J\rangle$': x.J.mean(), '$\sigma_J$': x.J.std()}))